In [1]:
using Intan, PyPlot

┌ Info: Precompiling Intan [160d5eed-cb0b-5d5e-9732-2f740c539f3e]
└ @ Base loading.jl:1342
ERROR: LoadError: ArgumentError: Package SpikeSorting [acc641c0-d14d-5ed3-8a01-2b4adfca2b15] is required but does not seem to be installed:
 - Run `Pkg.instantiate()` to install all recorded dependencies.

Stacktrace:
 [1] _require(pkg::Base.PkgId)
   @ Base .\loading.jl:1012
 [2] require(uuidkey::Base.PkgId)
   @ Base .\loading.jl:936
 [3] require(into::Module, mod::Symbol)
   @ Base .\loading.jl:923
 [4] include
   @ .\Base.jl:386 [inlined]
 [5] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::Nothing)
   @ Base .\loading.jl:1235
 [6] top-level scope
   @ none:1
 [7] eval
   @ .\boot.jl:360 [inlined]
 [8] eval(x::Expr)
   @ Base.MainInclude .\client.jl:446
 [9] top-level scope
   @ none:1
in expression starting at C:\Users\wanglab\Documents\Inta

LoadError: Failed to precompile Intan [160d5eed-cb0b-5d5e-9732-2f740c539f3e] to C:\Users\wanglab\.julia\compiled\v1.6\Intan\jl_11CD.tmp.

In [ ]:
#First create amplifier array:
myamp=RHD2164("PortA1")

#Task
mt=Task_TestTask()

#Save
mys=SaveNone()

#Initialize evaluation board setup
myrhd=makeRHD(myamp,"single",mt,sav=mys);

In [ ]:
handles = makegui(myrhd);

In [ ]:
function get_times(rhd,streams,samples,sr::Int64)
    
    myt=zeros(Float64,length(streams),length(samples))
    mytest=zeros(Float64,5)
    for s=1:length(streams)
        for j=1:length(samples)
            thebytes=(4+2(streams[s]*36)+8+2)*2*samples[j]
            for k=1:length(mytest)
                mytest[k]=@elapsed Intan.ReadFromPipeOut(rhd,Intan.PipeOutData,convert(Clong,thebytes),rhd.usbBuffer)   
            end
            myt[s,j]=mean(mytest)/(samples[j]/sr)
        end
    end
    
    myt.*100
end

In [ ]:
mysamples=collect(50:50:600)
sample_rate=20000
mystreams=collect(1:16)
t=get_times(myrhd,mystreams,mysamples,sample_rate);

In [ ]:
for i=1:size(t,1)
    plot(mysamples./sample_rate.*1000,t[i,:]',color="blue",alpha=i/20)
end

xlabel("Latency (ms)")
ylabel("% of processing window")

In [ ]:
for i=1:size(t,2)
    plot(collect(32:32:512),t[:,i],color="blue", alpha=i/20)
end
xlabel("Number of channels")
ylabel("% of Processing Window")